In [1]:
from neo4j import GraphDatabase
import ast
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import plot_confusion_matrix

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns


In [2]:
graphdb=GraphDatabase.driver(uri="bolt://localhost:7687",auth=("neo4j","kaushal")) #driver connects the graphdb and python 

start a session with browser

In [3]:
session = graphdb.session()

run_query returns a dataframe with quesry ans

In [4]:
def run_query(query, params={}):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result], columns=result.keys())

In [6]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [7]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="kaushal")

In [8]:
query="MATCH (n) return n LIMIT 30"
conn.query(query)

[<Record n=<Node id=0 labels=frozenset({'Client'}) properties={'id': '58744', 'c1': '1'}>>,
 <Record n=<Node id=1 labels=frozenset({'Client'}) properties={'id': '62913', 'c1': '1'}>>,
 <Record n=<Node id=2 labels=frozenset({'Client'}) properties={'id': '72066', 'c1': '1'}>>,
 <Record n=<Node id=3 labels=frozenset({'Client'}) properties={'id': '71679', 'c1': '1'}>>,
 <Record n=<Node id=4 labels=frozenset({'Client'}) properties={'id': '71749', 'c1': '1'}>>,
 <Record n=<Node id=5 labels=frozenset({'Client'}) properties={'id': '51458', 'c1': '1'}>>,
 <Record n=<Node id=6 labels=frozenset({'Client'}) properties={'id': '81962', 'c1': '1'}>>,
 <Record n=<Node id=7 labels=frozenset({'Client'}) properties={'id': '64345', 'c1': '1'}>>,
 <Record n=<Node id=8 labels=frozenset({'Client'}) properties={'id': '58762', 'c1': '1'}>>,
 <Record n=<Node id=9 labels=frozenset({'Client'}) properties={'id': '84098', 'c1': '1'}>>,
 <Record n=<Node id=10 labels=frozenset({'Client'}) properties={'id': '71147', '

create in-memory graphs with projection 

In [10]:
query = """CALL gds.graph.project(
  'tra',    
  ['Client', 'Transaction','Merchant'],   
  ['client_transaction', 'merchant_transaction']     
)
"""
conn.query(query)


Query failed: Failed to invoke procedure `gds.graph.project`: Caused by: java.lang.IllegalArgumentException: A graph with name 'tra' already exists.


run fastRP

In [19]:
def create_embs(dim=64):
    query="""
    CALL gds.fastRP.write(
    "tra",
    {
        embeddingDimension:%d,
        iterationWeights:[0.0,0.0,0.1],
        normalizationStrength:0.5,
        randomSeed:20,
        writeProperty: 'fastrp_embedding'
    }
)
    """ %(dim)
    conn.query(query)
    return
#create  x and target to train 
def create_X_y() : 
    query = "MATCH(g:Transaction) return ID(g) as id, g.fastrp_embedding AS fastrp_embedding"
    emb_df = pd.DataFrame([dict(_) for _ in conn.query(query)])
    emb_df['target'] = pd.factorize(emb_df['id'])[0].astype("float32")
    y = emb_df['target'].to_numpy()
    emb_df['X'] = emb_df['fastrp_embedding'].apply(lambda x: np.array(x))
    X = np.array(emb_df['X'].to_list())
    return X,y


create_model fumction trains data k folds  and gives indivisual accuracy scores of ll the folds along with average accuracy

In [20]:
def create_model(dim, k_folds=1, model='linear', show_matrix=True):

    acc_scores = []

    create_embs(dim=dim)
    X, y = create_X_y()

    for i in range(0, k_folds):
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
        clf = svm.SVC(kernel='linear', class_weight='balanced')
        clf.fit(X_train, y_train)
        pred = clf.predict(X_test)

        acc = accuracy_score(pred, y_test)
        acc_scores.append(acc)        
        
    print('Accuracy scores: ', acc_scores)
    print('Mean accuracy: ', np.mean(acc_scores))
    
    if show_matrix:
        matrix = plot_confusion_matrix(clf, X_test, y_test, cmap=plt.cm.Blues, normalize='true')
        plt.show(matrix)
        plt.show()
        
    return

call create_model

In [21]:
create_model(dim=64)


ValueError: Length of values (2) does not match length of index (1076)